In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import clean_text

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

In [6]:
%%time
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv_complet.csv")#delimiter=';',encoding='ISO-8859-1')
df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
#On complète les effets vide comme étant sans effets
df_declaration_mrv['TYPE_EFFET']  = df_declaration_mrv['TYPE_EFFET'].fillna("PAS D'EFFET NEFASTE DECLARE")
df_declaration_mrv['TEF_ID']= df_declaration_mrv['TEF_ID'].fillna('E1213')

#on selectionne les colonnes avec des effets
df = df_declaration_mrv[['DESCRIPTION_INCIDENT','TYPE_VIGILANCE','LIBELLE_COMMERCIAL',
                         'REFERENCE_COMMERCIALE','ETAT_PATIENT','FABRICANT','DCO_ID',
                         'ACTION_PATIENT','CLASSIFICATION','TEF_ID']]#[df_declaration_mrv['TEF_ID']!='E1213']
# On complète les NaN avec du vide
df['ETAT_PATIENT'] = df['ETAT_PATIENT'].fillna(" ")
df['DESCRIPTION_INCIDENT'] = df['DESCRIPTION_INCIDENT'].fillna(" ")
df['LIBELLE_COMMERCIAL'] = df['LIBELLE_COMMERCIAL'].fillna(" ")
df['FABRICANT'] = df['FABRICANT'].fillna(" ")
df["REFERENCE_COMMERCIALE"] = df['REFERENCE_COMMERCIALE'].fillna(" ")
df['TYPE_VIGILANCE'] = df['TYPE_VIGILANCE'].fillna(" ")
df['CLASSIFICATION'] = df['CLASSIFICATION'].fillna(' ')
df['DCO_ID'] = df['DCO_ID'].fillna(-1)
#On nettoieles variables textueelles : 

for col in  ['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','ETAT_PATIENT','FABRICANT','ACTION_PATIENT'] :
    df[col] = df[col].map(lambda x: clean_text.preprocess_text(x))


df['text'] = df['DESCRIPTION_INCIDENT']+'. '+df['LIBELLE_COMMERCIAL']+'. '+df['ETAT_PATIENT'] +'. '+df['FABRICANT']+'. '+df['ACTION_PATIENT']

features = 'text'
y_col = 'TEF_ID'

print(len(df))
n = 15
# On filtre pour a voir plus de n observations par classse
df = df.groupby("TEF_ID").filter(lambda x: len(x) > n)
print(len(df))
le_v = LabelEncoder()
df.TYPE_VIGILANCE = le_v.fit_transform(df.TYPE_VIGILANCE.values)
le = LabelEncoder()
df.TEF_ID = le.fit_transform(df.TEF_ID.values)

train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df, groups=df['DESCRIPTION_INCIDENT']))
df_train, df_test = df.iloc[train_index], df.iloc[test_index]
y = df_train.TEF_ID
y_test =df_test.TEF_ID

df_m = df.groupby('DESCRIPTION_INCIDENT')['TEF_ID'].apply(list).reset_index(name='multilabels')


df_ = pd.merge(df,df_m, on = 'DESCRIPTION_INCIDENT')
df_['multilabels'] = df_['multilabels'].apply(np.array)
df_['multilabels'] = df_['multilabels'].map(np.unique)

train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df_, groups=df['DESCRIPTION_INCIDENT']))
df_train, df_test = df_.iloc[train_index], df_.iloc[test_index]
y = df_train.TEF_ID
y_test =df_test.TEF_ID


y = df_.multilabels.iloc[train_index]
y_test =df_.multilabels.iloc[test_index]
lb = MultiLabelBinarizer()
y_lb =lb.fit_transform(y) 
y_test_lb = lb.transform(y_test)

76954
76402
CPU times: user 31.6 s, sys: 376 ms, total: 32 s
Wall time: 32 s


In [7]:
file_train = np.load('dgs_camembert_train.npz')
X_train = file_train['arr_0']
file_test =np.load('dgs_camembert_test.npz')
X_test = file_test['arr_0']
y_train = np.load('y_train_mlb.npy')
y_test = np.load('y_test_mlb.npy')

In [30]:
from sklearn.base import BaseEstimator, TransformerMixin


class CamenBertVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X,y=None):
        return self

    def transform(self, X):
        if type(X)==str:       
            return np.array(self.model.encode([X],show_progress_bar=True, batch_size=10))
        elif type(X)==list :     
            return np.array(self.model.encode(X,show_progress_bar=True, batch_size=10))
        elif type(X)==pd.Series :      
            return np.array(self.model.encode(X.values.tolist(),show_progress_bar=True, batch_size=10))
        else :
            print('Please enter a list of string or a pandas Series')
    
def get_name_effet(x:str)-> str :
    """
    Renvoie le nom de l'effet à partir  du numéro précisé dans le reférentiel
    si le numéro n'existe pas, alors il ne renvoi rien
    """
    try :
        return (df_effets[df_effets['TEF_ID']==int(x[1:])]['TYPE_EFFET'].iloc[0])
    except :
        return("")
    
from sentence_transformers import SentenceTransformer, models

word_embedding_model =  models.Transformer('./Models/Models/finetune2_nli_dgs_max_len_128_fp16_dynamic_padding_smart_batching_batch_32_seed_321/')

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [11]:
df_train = pd.DataFrame(X_train)
df_train['label'] = np.argmax(y_train, axis=1)

df_test = pd.DataFrame(X_test)
df_test['label'] = np.argmax(y_test, axis=1)

In [14]:
len(df_train.label.unique())

165

In [31]:
df_cat = pd.DataFrame()
df_cat['label'] = df_train.label.unique()
df_cat['label_code'] = df_cat['label'].map(lambda x : le.inverse_transform([x]))
df_cat['label_name'] = df_cat['label_code'].map(lambda x: get_name_effet(x[0]))

enc = CamenBertVectorTransformer(model = model)

df_cat['vect'] = df_cat['label_name'].map(lambda x : enc.transform(x))

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.12it/s]


In [32]:
df_cat

,label,label_code,label_name,vect
0,73,[E1213],PAS D'EFFET NEFASTE DECLARE,"[[-0.12912227, 0.24663861, 0.16318962, -0.1157..."
1,6,[E1111],ALLERGIE,"[[-0.027139882, 0.2573235, 0.18026677, -0.0884..."
2,82,[E1232],REACTION ALLERGIQUE,"[[0.07620083, 0.2766769, 0.19632113, -0.142262..."
3,111,[E1406],RÃIMPLANTATION CHIRURGICALE Ã RÃALISER,"[[-0.08031323, 0.24116795, 0.18124004, -0.0881..."
4,79,[E1220],PERTE DE SANG,"[[0.009299707, 0.22341165, 0.17176278, -0.1055..."
...,...,...,...,...
160,165,[E2277],EXPLANTATION CHIRURGICALE DU DM,"[[0.030921623, 0.26580817, 0.20100842, -0.1176..."
161,166,[E2279],CHIRURGIE ADDITIONNELLE,"[[-0.008396865, 0.27437043, 0.19115587, -0.129..."
162,172,[E2343],,"[[-0.12610511, 0.23436727, 0.18472506, -0.1131..."
163,169,[E2304],EMPLOI D'UN DM ALTERTIF,"[[-0.1556438, 0.24646418, 0.16714302, -0.08178..."


In [35]:
%%time
from sklearn.metrics.pairwise import cosine_similarity

def predict(x,cat_vect) :
    max_sim = 0
    winner = -1
    for j in cat_vect.index :
        sim = cosine_similarity(x,cat_vect.loc[j]['vect'].reshape(1, -1))
        if sim > max_sim :
            max_sim = sim
            winner = j
    return winner

P= []
for x in X_test : 
    P.append(predict(x.reshape(1, -1),df_cat))

df_test['pred'] = P

CPU times: user 39min 37s, sys: 4.21 s, total: 39min 41s
Wall time: 39min 36s


In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score,classification_report
print(f1_score(df_test['label'].values.tolist(),P,average='weighted'))

0.009887120745527413


In [46]:
a = enc.transform("le patient pleure")
b = enc.transform('le patient saigne')

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.00it/s]


In [47]:
sim = cosine_similarity(a.reshape(1, -1),b.reshape(1, -1))
print(sim)

[[0.9641523]]


In [45]:
df_cat[df_cat['label'] == predict(b.reshape(1, -1),df_cat)]

,label,label_code,label_name,vect
54,9,[E1115],BLESSURE,"[[-0.07799501, 0.23305276, 0.17006932, -0.0841..."


In [ ]:
df_cat